# TME 6 : Projet Detection de motifs


## Recheche de pattern (motifs) en utilisant les algoritmes randomisés

Les algorithmes randomisés prendre des décisions aléatoire plutôt que déterministes.
l'algorithme s'exécute différemment à chaque fois. Ils sont couramment utilisés dans situations où aucun algorithme exact et rapide est connu. Nous allons implémenter deux algorithmes randomisés pour la recherche de motifs : Greedy Profile Motif Search et random Projections.


1\. Nous allons réutiliser les fonctions du TME5 pour générer `t` séquences artificielles de taille `n`, et implanter dans chaque séquence un motif de taille `k` à des positions aléatoires avec `v` substitutions choisies aléatoirement.

In [1]:
import math
import random
from random import *
import operator
import numpy as np

nuc = ('A', 'C', 'G', 'T')

k=5 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence


In [2]:

def generateRandomSequence(n, upper=True):
    """
    Génère une séquence nucléotidique aléatoire 
    entrée n : longueur de la sequence
    entrée upper : bool, si True les nucléotides seront majuscule, False minuscule
    sortie sequence : une séquence nucléotidique aléatoire 
    """
    sequence = ""
    if upper:      
        for i in range(0,n) :
            sequence+= choice(nuc).upper()
    else:
        for i in range(0,n) :
            sequence+= choice(nuc).lower()
   
    return sequence



def posrand(taille, nb_nu):
    """
    taille : len du motif
    nb_nu : nb de nucleotides a changer
    choisi dans une sequence les positions où insérer les motifs
    renvoie une liste de int #position
    """
    listpos = []
    i = 0
    while i < nb_nu:
        rand = randint(0,taille-1)
        if rand not in listpos :
            listpos.append(rand)
            i += 1
    return sorted(listpos)


def modifierMotif(motif, nbpos,  upper=True):
    """
    Modifie nbpos positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée nbpos: nombre de positions
    entrée upper : bool, si True les nucléotides modifiés seront majuscule, False minuscule
    sortie motifM: motif modifié
    """
    motifM = ''
    #je transforme le motif en liste pour changer les nuc
    listemotif = list(motif)
    listePos = posrand(len(motif),nbpos)
    
    for n in listePos :
        listemotif[n] = choice(nuc)
        
    motifM = "".join(listemotif)
    
    if upper :
        motifM = motifM.upper()
    else :
        motifM = motifM.lower()

    return motifM


def implantMotifVar(k, v, t, n):
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée k: taille du motif
    entrée v: nombre de variations
    entrée t : nombre de séquences 
    entrée n : longueur des séquences
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus le motif doit être égal à n, pensez à générer de séquence aléatoire de taille n-k pour pouvoir implanter un motif de taille k
    """
  
    sequences = []
    taille = n-k
    motif = generateRandomSequence(k)
    
    print("Le motif : " + motif)
    for i in range(t):
        seq = generateRandomSequence(taille,True)
        
        listeseq = list(seq)
        listeseq.insert(randint(0,taille-1), modifierMotif(motif, v))
        seq = ''.join(listeseq)
        sequences.append(seq)
    

    return sequences

adn = implantMotifVar(k, v, t, n)
#print (adn)

adn  = [s.upper() for s in adn]
print (adn)

Le motif : GGCTG
['CAAAGACTGG', 'AGCGCTGATG', 'TGGCTTCGGT']


2\. Faire une fonction pour sélectionner des positions de départ aléatoirement `s = (s1, …,st)`.


In [3]:
#creating vector s
#Get t random positions from 0 to n - k
def generateRandomS(sequences, k):
    """
    Génère un vecteur de position aléatoires
    entrée sequences: matrice de dimension txn contenant les sequences
    entrée k: taille du motif
    sortie s: vecteur de position aléatoires, une position par séquence
    REMARQUE les positions doivent être inférieur à n-k
    """
    s = []
    for i in range(len(sequences)) : 
        s.append(randint(0,len(sequences[i])-k-1))
    return s

s = generateRandomS(adn, k)
print (adn)
print (s)

['CAAAGACTGG', 'AGCGCTGATG', 'TGGCTTCGGT']
[0, 4, 0]


3\. Extraire les motifs en utilisant le vecteur `s`, et construire un profile (matrice de fréquence). Attention, utiliser les pseudocount 1 pour éviter les plus tard les probabilités à zéro.


In [4]:
def extractSeqs(s, seqs, k):
    """
    Extraire les motifs des séquences à l'aide de positions s
    entrée s: vecteur contenant les positions de départs
    entrée seqs: matrice de dimension txn avec les séquences
    entrée k: taille du motif
    sortie motifs: liste de motifs
    """
    motifs = []   
    if (len(seqs) != len(s)):
        print("erreur len(seqs) != len(s) ")
        return
    i = 0
    while i < len(seqs):
        motifs.append(seqs[i][s[i]:s[i]+k])
        i = i+1
        
    return motifs

#Construire matrice de fréquence
def profile(motifs, k, nuc):
    """
    Construire une matrice de fréquence de dimension k x |nuc|
    entrée motifs: liste de motifs
    entrée k: taille du motif
    entrée nuc: alphabet
    sortie MF: matrice de fréquence
    """
    
    MF = np.ones((len(nuc), k))
    
    dico = {"A":0, "a":0, "C":1, "c":1, "G":2, "g":2, "T":3, "t":3}
    
    i = 0
    while i < k:
        j = 0
        while j < len(motifs):
            MF[dico[motifs[j][i]]][i] = MF[dico[motifs[j][i]]][i] + 1
            j=j+1
        i=i+1
        
    return MF

motifs = extractSeqs(s, adn, k)
print(motifs)
MF = profile(motifs, k, nuc)
print(MF)

['CAAAG', 'CTGAT', 'TGGCT']
[[1. 2. 2. 3. 1.]
 [3. 1. 1. 2. 1.]
 [1. 2. 3. 1. 2.]
 [2. 2. 1. 1. 3.]]


4\. Transformer la matrice de fréquence en PWM. Il faut diviser chaque élément par la somme de ses colonnes. Faire aussi une fonction pour calculer la sequence consensus d'une matrice de frequence.

In [5]:
motifs = [s.upper() for s in motifs]

def generatePWM(MF):
    """
    Transforme la matrice de fréquence en PWM
    entrée MF: matrice de fréquence
    sortie PWM: matrice de probabilité (poids positions)
    """
    somme = 0 
    for i in range(len(MF)) : 
        somme+= MF[i][0]

    PWM = np.zeros((len(MF), len(MF[0])))
    
    for i in range(len(MF)) : 
        for j in range(len(MF[i])) :
            PWM[i][j] = MF[i][j]/somme 
            
    return PWM

PWM = generatePWM(MF)

print(PWM)

[[0.14285714 0.28571429 0.28571429 0.42857143 0.14285714]
 [0.42857143 0.14285714 0.14285714 0.28571429 0.14285714]
 [0.14285714 0.28571429 0.42857143 0.14285714 0.28571429]
 [0.28571429 0.28571429 0.14285714 0.14285714 0.42857143]]


In [6]:
def getConsensus(MF, k, nuc):
    """
    Renvoie la sequence consensus d'une matrice de frequence
    entrée MF: matrice de fréquence
    entrée k: taille du motif
    entrée nuc: alphabet
    sortie seqCons: sequence consensus
    """
    dic = {0:"A", 1:"C", 2:"G", 3:"T"}
    
    seqCons = ""
    for i in range(k):
        pos = 0
        maxx = 0
        for j in range(4):
            if maxx < MF[j][i]:
                maxx = MF[j][i]
                pos = j
        seqCons += dic[pos]
        
    return seqCons

seqCons = getConsensus(MF, k, nuc)
print(seqCons)

CAGAT


5\. Faire une fonction pour calculer la probabilité d'un motif de taille `k` selon une PWM.


In [7]:
def probability(PWM, motif):
    """
    Calcul la probalité d'un motif selon PWM
    entrée PWM: matrice de probabilité (poids positions)
    entrée motif: motif
    sortie prob: probalité Prob(motif|PWM)
    """
    pos = 0; prob = 1
    #print (PWM)
    for n in motif:
        i = nuc.index(n)
        #print (i, pos)
        prob *=PWM[i, pos]
        pos +=1
    return prob

prob = probability(PWM, seqCons)
print (prob)

0.00963884095912417


6\. Faire une fonction pour calculer le pMostProbkmer d'une séquence, voir un exemple dans les slides de cours.

In [8]:
def pMostProbkmer(PWM, k, sequence):
    """
    calcul la position du k-mer le plus probable dans la séquence
    entrée PWM: matrice de probabilité (poids positions)
    entrée k: taille du motif
    entrée sequence: séquence nucleotidique
    sortie s: la position du k-mer le plus probable dans la sequence
    """
    
    s = 0 # position
    probmax = 0
    for i in range(len(sequence)-k) :
        Prob = probability(PWM, sequence[i:i+k])
        if (probmax < Prob) :
            s = i
            probmax = Prob
        
        
    return s

s = pMostProbkmer(PWM, k, "GCC" + seqCons+ "GGATACT")

print (s)

3


7\. Faire une fonction pour obtenir les nouvelles positions de départ `s = (s1, …,st)`, c’était à dire les positions qui contiens les k-mer le plus probables.

In [9]:
def getNewS(PWM, k, sequences):
    """
    Trouve les nouvelles positions des k-mer le plus probables
    entrée PWM: matrice de probabilité (poids positions)
    entrée k: taille du motif
    entrée sequence: séquence nucleotidique
    sortie s: vecteur avec les positions les plus problables
    """

    s = []
    
    for i in sequences :
        
        s.append(pMostProbkmer(PWM, k, i))
   
    return s
print(adn)
print(MF)
ns = getNewS(PWM, k, adn)
print (ns)

['CAAAGACTGG', 'AGCGCTGATG', 'TGGCTTCGGT']
[[1. 2. 2. 3. 1.]
 [3. 1. 1. 2. 1.]
 [1. 2. 3. 1. 2.]
 [2. 2. 1. 1. 3.]]
[0, 4, 0]


8\. La condition d’arrêt de l'algorithme est le non changement de la matrice de fréquence d'une itération à l'autre, 
faire une fonction pour comparer deux matrices et détecter le changement. 
Faire aussi une fonction pour obtenir le score d'une matrice de fréquence : la somme de max de chaque colonne

In [10]:
def changeProfile(P1, P2):
    """
    Compare deux matrice
    entrée P1: matrice de fréquence
    entrée P2: matrice de fréquence
    sortie: True si les matrices sont différents, False au contraire
    """
    
    return np.array_equal(P1, P2, equal_nan=True)

def getScore(MF, k):
    """
    Renvoie le score de MF, la somme des max de chaque colonne
    entrée P: matrice de fréquence
    entrée k: taille du motif
    sortie sc: score
    """
    sc = 0

    for i in range(k):
        maxx= 0
        for j in range(4):
            if maxx < MF[j][i]:
                maxx = MF[j][i]
        sc+= maxx
    
    return sc

ns = getNewS(PWM, k, adn)
print (s, ns)

sc = getScore(MF, k)
print (sc)


3 [0, 4, 0]
14.0


9\. Implementer l'algorithme ``GreedyProfileMotifSearch`` en utilisant les fonctions precedentes. 


In [11]:
def GreedyProfileMotifSearch(sequences, t, n, k):
    """
    Implémente l'algorithme GreedyProfileMotifSearch 
    entrée sequences: matrice de dimension txn contenant les séquences 
    entrée t : nombre de séquences 
    entrée n : longueur des séquences 
    entrée k : taille du motif 
    sortie s : vecteur de positions de départ ayant le meilleur motif 
    sortie bestScore : le score associé à s
    sortie seqCons : consensus
    
1. Sélectionnez des positions de départ aléatoirement s = (s 1 , ...,s t ).
2. Créez un profil Pr à partir des séquences de ces positions de départ.
3. Trouver le P-most-probable k-mers a pour chaque séquence et changer la position de
départ à la position de départ de a.
4. Calculer un nouveau profil basé sur le nouveaux positions de départs
5. procéder jusqu'à ce qu’on ne peut plus changer le Profil ou augmenter le score(s,
DNA)
    """
    vec = generateRandomS(sequences, k)
    #print(vec)
    motif1 = extractSeqs(vec, sequences, k)
    #print(motif1)
    MF1 = profile(motif1, k, nuc)
    #print(MF1)
    PWM1 = generatePWM(MF1)
    #print(PWM1)
    seqCons1 = getConsensus(MF1, k, nuc)
    #print(seqCons1)
    prob1 = probability(PWM1, seqCons1)
    #print(prob1)
    ns1 = getNewS(PWM1, k, sequences)
    #print(ns1)

    b = True
    while b :
        motif2 = extractSeqs(ns1, sequences, k)
        #print(motif2)
        MF2 = profile(motif2, k, nuc)
        #print(MF2)
        PWM2 = generatePWM(MF2)
        #print(PWM2)
        seqCons2 = getConsensus(MF2, k, nuc)
        #print(seqCons2)
        prob2 = probability(PWM2, seqCons2)
        #print(prob2)
        ns2 = getNewS(PWM2, k, sequences)
        #print(ns2) 
        #print(getScore(MF1, k))
        #print(getScore(MF2, k))
        
        if (not changeProfile(PWM1, PWM2) )or (getScore(MF1, k) > getScore(MF2, k)) : 
            MF1 = MF2
            PWM1 = PWM2
            seqCons1 = seqCons2
            prob1 = prob2
            ns1 = ns2
        else : 
            b = False 

    s = getNewS(PWM1, k, sequences)
    bestScore = getScore(MF1, k)
    
    return s, bestScore, seqCons1

s, bestScore, seqCons = GreedyProfileMotifSearch(adn, t, n, k)   
motifs = extractSeqs(s, adn, k)
print(seqCons)
print (motifs)
        

CAAAG
['CAAAG', 'CGCTG', 'CTTCG']


10\. Avez vous trouvez le motif implanté? Rexécuter l’algorithme plusieurs fois pour le trouver. 

Reponse: Notre motif de base CCCTG a pu être retrouvé en rexécutant plusieurs fois l'algorithme.

11\. Vous avez certainement observer que l’algorithme ne produire pas toujours la même sortie et que les résultats dépendent de la sélection aléatoire des positions de départ `s`. Pour augmenter nous chances de retrouvez les bons motifs, nous allons implémenter une version itérative ``GreedyProfileMotifSearchIte`` 
qui a chaque fois sauvegarde `s` et le score du profile associé à `s`, après `I` itération, l’algorithme renvoie le vecteur `s` ayant le plus grand score.

In [12]:


def GreedyProfileMotifSearchIte(sequences, t, n, k, It):
    """
    Implémente l'algorithme GreedyProfileMotifSearch iteratif
    entrée séquences: matrice de dimension txn contenant les séquences 
    entrée t : nombre de séquences 
    entrée n : longueur des séquences 
    entrée k: taille du motif 
    entrée It: nombre d'iterations
    sortie positions: dictionnaire clé=score, valeur= vecteur s 
    sortie consensus: dictionnaire clé=score, valeur= consensus sequence du motif
    sortie maxBestScore : la clef la plus élevée
    """
   
    pos = {}
    consensus = {}
    maxBestScore = 0
    
    i = 0
    while i < It :
        s, bestScore, seqCons = GreedyProfileMotifSearch(sequences, t, n, k)
        
        pos[bestScore] = s
        consensus[bestScore] = seqCons
        
        if maxBestScore < bestScore :
            maxBestScore = bestScore
        i+=1
    
    return pos, consensus, maxBestScore


pos, consensus, maxBestScore = GreedyProfileMotifSearchIte(adn, t, n, k, 50)
print(pos)
print(consensus)
print(pos[maxBestScore])

{15.0: [3, 0, 0], 12.0: [2, 2, 2], 14.0: [2, 1, 2], 13.0: [1, 3, 1], 17.0: [4, 2, 1], 11.0: [1, 1, 4], 16.0: [3, 1, 0]}
{15.0: 'AGACT', 12.0: 'AACTC', 14.0: 'GCGAC', 13.0: 'GAAGA', 17.0: 'GGCTG', 11.0: 'AAAGA', 16.0: 'AGGCT'}
[4, 2, 1]


12\. Tester algorithme  ``GreedyProfileMotifSearchIte`` sur vos données de chipSeq. N'oubliez pas de chercher les motifs dans le brin complémentaire et faire un merge de résultats.

In [14]:
k=7; v=1; t=20; n=80

def reversecompl(seq):
    """Renvoie le brin complémentaire d’une séquence.
    entrée : sequence de nucléotides (brin sens)
    sortie : sequence de nucléotides (brin complementaire)
    >>> reversecompl('AACGTGGCA')
    'TGCCACGTT'
    """
    compl = {'A': 'T', 'C': 'G', 'G': 'C', 'T':'A','a': 't', 'c': 'g', 'g': 'c', 't':'a'}
    
    l = list(seq)
    for i in range(len(seq)) : 
        l[i] = compl[seq[i]]
    l.reverse()
    return ''.join(l)

def readFasta(genome, n):
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    sequenceStr = "".join(sequence)
    sequence = [sequenceStr[i:i+n] for i in range(0, len(sequenceStr), n)]
    sequenceRet = [x for x in sequence if x]
    return sequenceRet

genome = "Sequence_by_Peaks_2.fasta" #Remplacer par votre fichier



sequencesChip = readFasta(genome, n)

chipSequences = []
for s in sequencesChip : 
    chipSequences.append(reversecompl(s)) 
    
t = len(sequencesChip)
pos, consensus, maxBestScore = GreedyProfileMotifSearchIte(sequencesChip, t, n, k, 50)
posComp, consensusComp, maxBestScoreComp = GreedyProfileMotifSearchIte(chipSequences, t, n, k, 50)

""" donne liste des motifs trouvés sur les deux brins :"""

motifs = extractSeqs(pos[maxBestScore], sequencesChip, k)
motifsComp = extractSeqs(posComp[maxBestScoreComp], chipSequences, k)

mergeMotifs = list(set(motifs + motifsComp))
print(mergeMotifs)

['TCCTTTG', 'TGAATCT', 'TAGTTTA', 'TGCAGTA', 'AGAATTG', 'GGAAGCT', 'TTGGGTG', 'TTTTTTG', 'TTGTCTC', 'TATCATC', 'TACCCGG', 'TATCCTC', 'TTGTATC', 'AGAAACG', 'TGCAGCT', 'TTACATA', 'ACAAATT', 'GGAAGAG', 'TAATTTC', 'TTGCTGG', 'TACACGC', 'TTTTAGC', 'TGAAGTG', 'TAACCTC', 'TGAACAT', 'TCGCTTG', 'TTCCAGA', 'TTCTTGA', 'AACAATT', 'TCCAATC', 'TTCATTC', 'TGCAGAA', 'TTATGGG', 'TACTAGC', 'AGAAACT', 'TACTATG', 'TACTCGG', 'TTGCTTG', 'TTGCAGG', 'GGTAATT', 'TCAACTT', 'TATTTTG', 'TTCCCTC', 'TTGGTTA', 'TTTCTGA', 'TTACCTC', 'TACTTTC', 'GGAAAAG', 'AGAAGAT', 'TATAAAT', 'TTGGATG', 'TTTTATC', 'AAAAATT', 'TTCTTTA', 'TTGTCGC', 'TTCTGTA', 'TTGGTGA', 'TGCAACA', 'TGTACAG', 'TGTAAAT', 'TCGCTTC', 'TACATTA', 'TCTAATG', 'AGTAGAT', 'AGAACCC', 'TTCCCTA', 'TTGGATA', 'TACGCTG', 'GGAACTG', 'TGTATTT', 'TTATATC', 'TACCTTA', 'AGAACAA', 'AACAATA', 'TAAAGTA', 'TGTAGTG', 'TGCAAAG', 'TTGGCTG', 'AGCAGCG', 'AGAAATA', 'TTTGCTG', 'AGAAGAG', 'TTATATG', 'TGAATTG', 'TGCAGCG', 'TACTCTG', 'TTCTAGG', 'AGAAAAC', 'TTCTCTA', 'GGCAGTT', 'AGAAGAA'

Afficher les top motifs (leurs sequence consensus) de meilleurs scores.

In [15]:
def printTopMotifsScore(consensus, top):

    motifsSort = sorted(consensus.keys())
    motifsSort.reverse()
    tailledico = len(consensus)
    i = 0
    while i<top and i<tailledico :
        print (consensus[motifsSort[i]])
        i+=1

top = 10
printTopMotifsScore(consensus, top)
print("________")
printTopMotifsScore(consensusComp, top)

TTCTTTC
TATTTCT
TTTCTTT
AATTGCT
AAGAAGT
TTTTTCT
CTTCTTG
CAATTTT
TTTCTCT
TTTCTTT
________
AGAAATT
AGAGAAA
AGAAACT
AGAAATA
AGAGAAA
ATGGAAA
TGGAAAT
AGAGAAA
AAATTCT
AAAGAAA


13\. Nous allons implémenter l'algorithme ``randomProjection``. D'abord, faites la fonction `getRandomFixePositions` pour générer une projection de k à n, voir un exemple dans les slides de cours. Faire aussi la fonction `generateKey` qui extrait les caractères du motif puis génère une cle qui représente la projection.

In [16]:
def getRandomFixePositions(k, n):
    """
    Genere une projection de k vers n
    entrée k: nombre de positions du motif original
    entrée n: nombre de positions choisi pour la projection 
    sortie projection: liste de positions choisi aléatoirement
    """
    return posrand(k,n)

lR = getRandomFixePositions(7, 4)
print (lR)

def generateKey(positions, motif):
    """
    extrait les caractères du motif et génère la cle de la projection
    entrée positions : liste de positions qui represent la projection
    entrée motif : motif de taille k
    sortie cle : cle de la projection
    """
    cle = ""
    
    for i in positions :
        cle+= motif[i]

    return cle

print(generateKey(lR, "0123456"))

[1, 3, 5, 6]
1356


14\. Implémenter l'algorithme ``randomProjection``.

In [17]:
def randomProjection(k, n, sequences):
    """
    Implémente l'algorithme randomProjection
    entrée k : nombre de positions du motif
    entrée n : nombre de positions de la projection 
    entrée sequences : matrice de dimension txn contenant les séquences 
    sortie motifs : dictionaire, cle = projection, valeur= frequence
    sortie motifsSeq:  dictionaire, cle = projection, valeur= original motif
    """
    motifs  = {}; motifsSeq  = {}
    for s in sequences : 
        pro = generateKey(getRandomFixePositions(k,n), s)
        if pro not in motifs :
            motifs[pro] = 0
            motifsSeq[pro] = []
        motifs[pro] += 1
        motifsSeq[pro].append(s)
        
    return motifs, motifsSeq


#motifsSort = sorted(motifs, reverse=True, key=motifs.get)

adnTest = ['TTAACGGCAC', 'GCTCACGCCA', 'TACCGGCCGT']
motifsProj, motifsSeq = randomProjection(7, 4, adnTest)
print (motifsProj)
print (motifsSeq)


{'TTCG': 1, 'CTCC': 1, 'TAGC': 1}
{'TTCG': ['TTAACGGCAC'], 'CTCC': ['GCTCACGCCA'], 'TAGC': ['TACCGGCCGT']}


15\. Avez vous trouvez le motif implanté? Rexécuter l’algorithme plusieurs fois pour augmenter les chances de le trouver. 

reponse: Le motif implanté pourrait être TACG car il a une fréquence de 2, dans le cas où un vecteur de position différent est utilisé pour chaque séquence. 

16\. Implémenter la version itérative de l’algorithme ``randomProjection``. 

In [18]:
def randomProjIt(sequences, k, v, nuc, It):
    """
    Implémente l'algorithme randomProjection version iteractive
    entrée sequences : matrice de dimension txn contenant les séquences 
    entrée k : nombre de positions du motif
    entrée v : nombre de positions de la projection 
    entrée nuc : alphabet
    entrée It: nombre d'iterations
    sortie score : meilleur score
    sortie motifs :  liste de motifs associés au meilleur score
    """
    
    motifs = []; scores = []
    
    i=0
    while i < It : 
        motifsProj, motifsSeq = randomProjection(k, v, sequences)
        for key, value in motifsSeq.items() : 
            motifs.extend(value)
        for key, value in  motifsProj.items() : 
            scores.append(value)
        
        i+=1
        
    return max(scores), motifs[scores.index(max(scores))] 

score, seqsMotif = randomProjIt(adn, 7, 4, nuc, 100)

print (score, seqsMotif)


1 CAAAGACTGG


17\. Tester l'algorithme  ``randomProjection`` sur vos données de chipSeq. N'oubliez pas de chercher les motifs dans le brin complémentaire et faire un merge de résultats. Puis générér le LOGO du motif trouvé.

In [19]:
score, seqsMotif = randomProjIt(sequencesChip, 7, 4, nuc, 100)
print (score, seqsMotif)

12 TTTCAGGCCTTGTTTCTTTCAAATTTTCAGCCAATAGGCGGCGAGTAGCAGCGCTGGGCGAGGATAATAGCACAATTTGA


18\. Vous avez sans doute remarqué que les algos implémentés trouvent souvent des motifs peu complexes, réviser les implémentations pour régler/diminuer ce problème.

Pour réduire le problème, on pourrait ignorer les motifs qui ont des nucléotides qui se répétent.
